Installing required libraries


In [160]:
%pip install -U scikit-learn
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Voldana\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Voldana\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Voldana\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Importing required stuff

In [161]:
import numpy as np
import pandas as pd 

Setting our dataset path

In [162]:
dataset_folder = r"D:\University Courses\Computational AI\Datasets\Final"
lfw_folder = r"D:\University Courses\Computational AI\Datasets\Final\lfw"

In [195]:
def extract_feature(fileName,index):
    
    path = lfw_folder+"\\" +fileName+"\\"+fileName+"_"+change_name_format(index)+".txt"    
    raw_data = replace_signs(path).split()
    arr = np.zeros((1,512))
    for i in range(0,512):
        arr[0][i] = np.float32(raw_data[i])
    return arr

In [194]:
def replace_signs(path):
    file=open(path).readlines()
    file=str(file).replace(r"\n', '  "," ")
    file=file.replace(",","")
    file=file.replace("'","")
    file=file.replace("[","")
    file=file.replace("]","")

    return file

In [193]:
def change_name_format(index):
    temp_num_0 = int(index / 1000)
    temp_num = index % 1000
    temp_num_1 = int(temp_num / 100)
    temp_num = temp_num % 100
    temp_num_2 = int(temp_num / 10)
    temp_num_3 = temp_num % 10 
    new_str = "{}{}{}{}".format(temp_num_0,temp_num_1,temp_num_2,temp_num_3)
    
    return new_str

Making a data set and a label set for both train and test sets

In [255]:
def make_set(data_list,number_of_classes):
    data_set=np.zeros((number_of_classes * 2,1024),dtype=np.float32)
    label_set=np.zeros(number_of_classes * 2,dtype=int)
    # make_subsets(data_list,data_set,label_set,0,number_of_classes,1)
    # make_subsets(data_list,data_set,label_set,number_of_classes + 1,number_of_classes,0)
    for i in range(0,number_of_classes):
        temp_str = data_list[i]
        temp_str = temp_str.replace("\n","\t")
        splitted = temp_str.split("\t")
        first_feature=extract_feature(splitted[0],int(splitted[1]))
        second_feature=extract_feature(splitted[0],int(splitted[2]))
        data_set[i] = np.concatenate((first_feature,second_feature),axis=1)
        label_set[i] = 1

    for i in range(number_of_classes,number_of_classes * 2):
        temp_str = data_list[i]
        temp_str = temp_str.replace("\n","\t")
        splitted = temp_str.split("\t")
        first_feature=extract_feature(splitted[0],int(splitted[1]))
        second_feature=extract_feature(splitted[2],int(splitted[3]))
        data_set[i] = np.concatenate((first_feature,second_feature),axis=1)
        label_set[i] = 0  
        
    return data_set,label_set

In [233]:
def make_subsets(data_list,data_set,label_set,start,number_of_classes,value):
    for i in range(start,start + number_of_classes - 5):
        temp_str = data_list[i]
        temp_str = temp_str.replace("\n","\t")
        splitted = temp_str.split("\t")
        first_feature=extract_feature(splitted[0],int(splitted[1]))
        second_feature=extract_feature(splitted[0],int(splitted[2]))
        data_set[i] = np.concatenate((first_feature,second_feature),axis=1)
        label_set[i] = value 

Making test and train sets

In [257]:
train_list = open(dataset_folder+"\\pairsDevTrain.txt").readlines()
train_class_count = train_list.pop(0)
train_data_set,train_label_set=make_set(train_list,int(train_class_count))

In [258]:
test_list = open(dataset_folder+"\\pairsDevTest.txt").readlines()
test_class_count = test_list.pop(0)
test_data_set,test_label_set=make_set(test_list,int(test_class_count))

Importing needed libraries

In [169]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

Testing bootsrap bagging with DT classifier

In [149]:
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=10).fit(train_data_set, train_label_set)

Checking bootstrap accuracy

In [150]:
bagging_pred=bagging_clf.predict(test_data_set)
bagging_acc=accuracy_score(bagging_pred,test_label_set)
print(bagging_acc)

0.578


Gonna try increasing the max_features but this might cause overfitting

In [151]:
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=10,max_features=512).fit(train_data_set, train_label_set)

In [152]:
bagging_pred=bagging_clf.predict(test_data_set)
bagging_acc=accuracy_score(bagging_pred,test_label_set)
print(bagging_acc)

0.579


So even if we use all of the features it doesn't improve accuracy by that much

ok let's increase the number of estimators and keep the max_features low

In [153]:
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=1000,max_features=100).fit(train_data_set, train_label_set)

In [111]:
bagging_pred=bagging_clf.predict(test_data_set)
bagging_acc=accuracy_score(bagging_pred,test_label_set)
print(bagging_acc)

0.577


Accuracy improved by 4% or so, let's try 500 estimators with default modes on max_samples and max_features with 8 as random state

... this took longer than i expected to run it took 11 minutes

In [20]:
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=500, random_state=8).fit(train_data_set, train_label_set)

In [21]:
bagging_pred=bagging_clf.predict(test_data_set)
bagging_acc=accuracy_score(bagging_pred,test_label_set)
print(bagging_acc)

0.712


We managed to increase the accuracy by about 13% so we might be on the right track the only problem is training time takes almost 100 times more than before so here I'm gonna try using the default values for max_sample and max_features but more estimators lets say 1000

In [22]:
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=1000, random_state=8).fit(train_data_set, train_label_set)

In [23]:
bagging_pred=bagging_clf.predict(test_data_set)
bagging_acc=accuracy_score(bagging_pred,test_label_set)
print(bagging_acc)

0.72


Ok after 23 minutes of training we only say 0.8% improvement in our accuracy so let's leave the bagging with DT classifier at here, we might be able to get more accuracy by changing the parameters but i don't think that it'd be that significant.. however this is the best accuracy we managed to reach so far

Let's try bagging with SVM classifier this time we're gonna use 100 max samples and 100 for max features and 10 estimators

In [26]:
svm_clf = BaggingClassifier(base_estimator=SVC(),n_estimators=10,max_samples=100,max_features=100).fit(train_data_set, train_label_set)

In [27]:
svm_pred=svm_clf.predict(test_data_set)
svm_acc=accuracy_score(svm_pred,test_label_set)
print(svm_acc)

0.558


Ok that's not very satisfying so let's try increasing the estimators and leaving max saple and max features to default

In [28]:
svm_clf = BaggingClassifier(base_estimator=SVC(),n_estimators=100).fit(train_data_set, train_label_set)

In [29]:
svm_pred=svm_clf.predict(test_data_set)
svm_acc=accuracy_score(svm_pred,test_label_set)
print(svm_acc)

0.729


Ok that's a huge improvement and better than our DT classifier by a littSle bit let's try lowering the estimators again

In [38]:
svm_clf = BaggingClassifier(base_estimator=SVC(),n_estimators=20).fit(train_data_set, train_label_set)

In [40]:
svm_pred=svm_clf.predict(test_data_set)
svm_acc=accuracy_score(svm_pred,test_label_set)
print(svm_acc)

0.729


Ok changing the estimators didn't lead to anything
Now let's change the SVM gamma to see if we can get an improvement in accuracy

In [72]:
svm_clf = BaggingClassifier(base_estimator=SVC(gamma = 0.0003),n_estimators=20).fit(train_data_set, train_label_set)

In [73]:
svm_pred=svm_clf.predict(test_data_set)
svm_acc=accuracy_score(svm_pred,test_label_set)
print(svm_acc)

0.754


Ok the improvement is decent at this point let's try different gamma values

In [78]:
svm_clf = BaggingClassifier(base_estimator=SVC(gamma = 0.0002),n_estimators=20).fit(train_data_set, train_label_set)

In [79]:
svm_pred=svm_clf.predict(test_data_set)
svm_acc=accuracy_score(svm_pred,test_label_set)
print(svm_acc)

0.764


That's the best accuracy we've been able to reach so 76.4% 

Let's try bagging with MLPs

In [170]:
from sklearn.neural_network import MLPClassifier

In [171]:
mlp_clf = BaggingClassifier(base_estimator= MLPClassifier(),n_estimators=10).fit(train_data_set, train_label_set)

In [172]:
mlp_pred=mlp_clf.predict(test_data_set)
mlp_acc=accuracy_score(mlp_pred,test_label_set)
print(mlp_acc)

0.761


Ok looks like we're on to a good start with MLP let's increase the number of estimators

In [173]:
mlp_clf = BaggingClassifier(base_estimator= MLPClassifier(),n_estimators=20).fit(train_data_set, train_label_set)

In [174]:
mlp_pred=mlp_clf.predict(test_data_set)
mlp_acc=accuracy_score(mlp_pred,test_label_set)
print(mlp_acc)

0.774


That's better than our SVM classifier since increasing estimators improved accuracy by 1.5% we're gonna keep doing that

In [175]:
mlp_clf = BaggingClassifier(base_estimator= MLPClassifier(),n_estimators=30).fit(train_data_set, train_label_set)

In [176]:
mlp_pred=mlp_clf.predict(test_data_set)
mlp_acc=accuracy_score(mlp_pred,test_label_set)
print(mlp_acc)

0.786


Increasing number of estimators is improving the accuracy.. however it's taking alot of CPU 

In [177]:
mlp_clf = BaggingClassifier(base_estimator= MLPClassifier(),n_estimators=50).fit(train_data_set, train_label_set)

In [178]:
mlp_pred=mlp_clf.predict(test_data_set)
mlp_acc=accuracy_score(mlp_pred,test_label_set)
print(mlp_acc)

0.782


Ok seems like increasing the number of estimators no longer improves accuracy, MLP might be the algorithem that gives us the best accuracy with bagging however it takes alot of my system's resources so i can't modify the parameters that much 

Lets Try Random forest to see what it has to offer us

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
rf_clf = RandomForestClassifier(n_estimators=10).fit(train_data_set,train_label_set)

In [46]:
rf_pred=rf_clf.predict(test_data_set)
rf_acc=accuracy_score(rf_pred,test_label_set)
print(rf_acc)

0.56


Lets increase the number of estimators

In [55]:
rf_clf = RandomForestClassifier(n_estimators=100).fit(train_data_set,train_label_set)

In [56]:
rf_pred=rf_clf.predict(test_data_set)
rf_acc=accuracy_score(rf_pred,test_label_set)
print(rf_acc)

0.631


Ok we saw a decent improvement by increasing the number of estimators so let's do that again

In [57]:
rf_clf = RandomForestClassifier(n_estimators=500).fit(train_data_set,train_label_set)

In [58]:
rf_pred=rf_clf.predict(test_data_set)
rf_acc=accuracy_score(rf_pred,test_label_set)
print(rf_acc)

0.687


ok 5% isn't that bad we might be able to reach more accuracy than bagging with SVM so let's add more estimators

In [59]:
rf_clf = RandomForestClassifier(n_estimators=5000).fit(train_data_set,train_label_set)

In [60]:
rf_pred=rf_clf.predict(test_data_set)
rf_acc=accuracy_score(rf_pred,test_label_set)
print(rf_acc)

0.73


Increasing the number of estimators improved random forest accuracy by around 4% so let's try giving it more estimators

In [104]:
rf_clf = RandomForestClassifier(n_estimators=10000).fit(train_data_set,train_label_set)

In [105]:
rf_pred=rf_clf.predict(test_data_set)
rf_acc=accuracy_score(rf_pred,test_label_set)
print(rf_acc)

0.726


Ok it seems random forest accuracy caps at 73% for us which is still less than our bagging method with SVM let's move on to the next method of ensemble learning

In [106]:
from sklearn.ensemble import AdaBoostClassifier

In [121]:
ada_boost_clf = AdaBoostClassifier().fit(train_data_set,train_label_set)

In [123]:
ada_boost_pred=ada_boost_clf.predict(test_data_set)
ada_boost_acc=accuracy_score(ada_boost_pred,test_label_set)
print(ada_boost_acc)

0.522


Let's increasing the number of estimators to see if it leads to any improvements

In [124]:
ada_boost_clf = AdaBoostClassifier(n_estimators = 500).fit(train_data_set,train_label_set)

In [125]:
ada_boost_pred=ada_boost_clf.predict(test_data_set)
ada_boost_acc=accuracy_score(ada_boost_pred,test_label_set)
print(ada_boost_acc)

0.531


The accuracy didn't change much by increasing the estimator numbers so let's increasing max depth of our DT

In [127]:
ada_boost_clf = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 2)).fit(train_data_set,train_label_set)

In [128]:
ada_boost_pred=ada_boost_clf.predict(test_data_set)
ada_boost_acc=accuracy_score(ada_boost_pred,test_label_set)
print(ada_boost_acc)

0.6


Well since the improvement is quite decent let's try increasing max_depth again to see what happens

In [130]:
ada_boost_clf = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3)).fit(train_data_set,train_label_set)

In [131]:
ada_boost_pred=ada_boost_clf.predict(test_data_set)
ada_boost_acc=accuracy_score(ada_boost_pred,test_label_set)
print(ada_boost_acc)

0.589


Increasing max_depth might have caused overfitting in this case

Bagging with DT : Best results were 72% accuracy using 1000 estimators and 8 random states
------------
Bagging with SVM: Best results were 76.4% accuracy using 0.0002 as gamma value and 20 estimators
------------
Bagging with MLP: Best results were 78.6% accuracy using 30 estimators
------------
Random Forest: Best results were 73% accuracy using 5000 estimators
------------
AdaBoosting : best results were 60% accuracy using a DT classifier with max_depth of 2


For now bagging using MLP classifier seems like the best option we have it has better accuracy than others also it takes barely any time to test on the test data unlike bagging with SVM which takes equal time to train or to test
Overall the accuracy of the mentioned algorithems can be improved by testing different configurations (specially for those using DT classifier)